In [1]:
!pip install onnxmltools

In [3]:
import numpy as np
import torch
import pandas as pd
import sklearn
import random

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from mlxtend.plotting import heatmap
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import r2_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix


In [4]:
import onnxmltools

In [5]:
iris_data = pd.read_csv('iris.csv')

In [6]:
print(iris_data.head())


   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [7]:
batch_size = 16
learning_rate = 0.001
N_Epochs = 100
epsilon = 0.0001

In [8]:
iris_data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [9]:
iris_data.species = iris_data.species.map( {'setosa':0 , 'virginica':1, 'versicolor':2} )


In [10]:
## Convert Pandas to Numpy
iris_data_np = iris_data.to_numpy()

In [11]:
iris_data_np


array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

In [12]:
X = iris_data_np[:, 0:-1]


y = iris_data_np[:, 4]

In [13]:
y = y.astype(int)

In [14]:
print(X.shape)

print(y.shape)

(150, 4)
(150,)


In [15]:
random_seed = int(random.random() * 100)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

In [17]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120, 4)
(30, 4)
(120,)
(30,)


In [18]:
X_train = X_train.astype(  np.float32  )
X_test  = X_test.astype(   np.float32  )
y_train = y_train.astype(  np.int64  )
y_test  = y_test.astype(   np.int64  )

In [19]:
X_train_tr = torch.from_numpy(X_train)
X_test_tr  = torch.from_numpy(X_test)
y_train_tr = torch.from_numpy(y_train)
y_test_tr  = torch.from_numpy(y_test)

In [20]:
#Standardization
x_means = X_train_tr.mean(0, keepdim=True)
x_deviations = X_train_tr.std(0, keepdim=True) + epsilon

In [21]:
train_ds = TensorDataset(X_train_tr, y_train_tr)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [22]:
class DL_Net(nn.Module):
    def __init__(self, x_means, x_deviations):
        super().__init__()
        self.x_means = x_means
        self.x_deviations = x_deviations
        self.linear1 = nn.Linear(4, 10)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(10, 6)
        self.act2 = nn.ReLU()
        self.linear3 = nn.Linear(6, 3)  # 3 classes (setosa, versicolor, virginica)

    def forward(self, x):
        x = (x - self.x_means) / self.x_deviations
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x)
        y_pred = self.linear3(x)
        return y_pred

In [23]:
def training_loop(N_Epochs, model, loss_fn, opt):
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            y_pred = model(xb)
            loss = loss_fn(y_pred, yb)
            opt.zero_grad()
            loss.backward()
            opt.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}: loss={loss.item()}")

In [24]:
##train model
model = DL_Net(x_means, x_deviations)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [25]:
training_loop(N_Epochs, model, loss_fn, opt)


Epoch 0: loss=1.1255624294281006
Epoch 20: loss=0.6822565197944641
Epoch 40: loss=0.42240339517593384
Epoch 60: loss=0.060467563569545746
Epoch 80: loss=0.1389244645833969


In [26]:
def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('F1-measure: %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

In [30]:
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tr)
    y_pred_labels = torch.argmax(y_pred_test, dim=1)

In [31]:
print_metrics_function(y_test_tr.numpy(), y_pred_labels.numpy())


Accuracy: 0.97
Confusion Matrix:
[[13  0  0]
 [ 0  8  0]
 [ 0  1  8]]
Precision: 0.970
Recall: 0.967
F1-measure: 0.967


In [32]:
# --- Export Model to ONNX ---

dummy_input = torch.randn(1, 4)
input_names = ["input1"]
output_names = ["output1"]

torch.onnx.export(
    model,
    dummy_input,
    "DLnet_IrisData.onnx",
    verbose=False,
    input_names=input_names,
    output_names=output_names
)

